# sales_data  - data cleaning and preprocessing

In [8]:
import pandas as pd

In [9]:
# Load dataset
file_path = 'sales_data.csv'  
df = pd.read_csv(file_path)

In [10]:
df.head()

,Product_ID,Sale_Date,Sales_Rep,Region,Sales_Amount,Quantity_Sold,Product_Category,Unit_Cost,Unit_Price,Customer_Type,Discount,Payment_Method,Sales_Channel,Region_and_Sales_Rep
0,1052,2023-02-03,Bob,North,5053.97,18,Furniture,152.75,267.22,Returning,0.09,Cash,Online,North-Bob
1,1093,2023-04-21,Bob,West,4384.02,17,Furniture,3816.39,4209.44,Returning,0.11,Cash,Retail,West-Bob
2,1015,2023-09-21,David,South,4631.23,30,Food,261.56,371.40,Returning,0.20,Bank Transfer,Retail,South-David
3,1072,2023-08-24,Bob,South,2167.94,39,Clothing,4330.03,4467.75,New,0.02,Credit Card,Retail,South-Bob
4,1061,2023-03-24,Charlie,East,3750.20,13,Electronics,637.37,692.71,New,0.08,Credit Card,Online,East-Charlie


#### Checking non-null count ,datatypes and missing values

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Product_ID            1000 non-null   int64  
 1   Sale_Date             1000 non-null   object 
 2   Sales_Rep             1000 non-null   object 
 3   Region                1000 non-null   object 
 4   Sales_Amount          1000 non-null   float64
 5   Quantity_Sold         1000 non-null   int64  
 6   Product_Category      1000 non-null   object 
 7   Unit_Cost             1000 non-null   float64
 8   Unit_Price            1000 non-null   float64
 9   Customer_Type         1000 non-null   object 
 10  Discount              1000 non-null   float64
 11  Payment_Method        1000 non-null   object 
 12  Sales_Channel         1000 non-null   object 
 13  Region_and_Sales_Rep  1000 non-null   object 
dtypes: float64(4), int64(2), object(8)
memory usage: 109.5+ KB


In [12]:
print(df.isnull().values.any())

False


#### Convert Sale_Date object type to datetime


In [13]:
df['Sale_Date'] = pd.to_datetime(df['Sale_Date'], errors='coerce')

In [14]:
df['Sale_Date'].dtype

dtype('<M8[ns]')

In [15]:
## date formats to a consistent type (dd-mm-yyyy)
df['Sale_Date'] = df['Sale_Date'].dt.strftime('%d-%m-%Y')
print(df)

     Product_ID   Sale_Date Sales_Rep Region  Sales_Amount  Quantity_Sold  \
0          1052  03-02-2023       Bob  North       5053.97             18   
1          1093  21-04-2023       Bob   West       4384.02             17   
2          1015  21-09-2023     David  South       4631.23             30   
3          1072  24-08-2023       Bob  South       2167.94             39   
4          1061  24-03-2023   Charlie   East       3750.20             13   
..          ...         ...       ...    ...           ...            ...   
995        1010  15-04-2023   Charlie  North       4733.88              4   
996        1067  07-09-2023       Bob  North       4716.36             37   
997        1018  27-04-2023     David  South       7629.70             17   
998        1100  20-12-2023     David   West       1629.47             39   
999        1086  16-08-2023     Alice   East       4923.93             48   

    Product_Category  Unit_Cost  Unit_Price Customer_Type  Discount  \
0   

#### Convert object datatype to category datatype

In [16]:
categorical_cols = ['Sales_Rep', 'Region', 'Product_Category', 'Customer_Type', 'Payment_Method', 'Sales_Channel','Region_and_Sales_Rep']
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [17]:
 df.dtypes

Product_ID                 int64
Sale_Date                 object
Sales_Rep               category
Region                  category
Sales_Amount             float64
Quantity_Sold              int64
Product_Category        category
Unit_Cost                float64
Unit_Price               float64
Customer_Type           category
Discount                 float64
Payment_Method          category
Sales_Channel           category
Region_and_Sales_Rep    category
dtype: object

In [18]:
df.info() ##memory usage reduce by converting into categorical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Product_ID            1000 non-null   int64   
 1   Sale_Date             1000 non-null   object  
 2   Sales_Rep             1000 non-null   category
 3   Region                1000 non-null   category
 4   Sales_Amount          1000 non-null   float64 
 5   Quantity_Sold         1000 non-null   int64   
 6   Product_Category      1000 non-null   category
 7   Unit_Cost             1000 non-null   float64 
 8   Unit_Price            1000 non-null   float64 
 9   Customer_Type         1000 non-null   category
 10  Discount              1000 non-null   float64 
 11  Payment_Method        1000 non-null   category
 12  Sales_Channel         1000 non-null   category
 13  Region_and_Sales_Rep  1000 non-null   category
dtypes: category(7), float64(4), int64(2), object(1)
memory us

#### check if standardizing text values is required


In [19]:
def check_text_standardization(df):
    issues = {}
    text_cols = df.select_dtypes(include=['object', 'string']).columns
    
    for col in text_cols:
        col_issues = []
        if df[col].str.lower().nunique(dropna=True) != df[col].nunique(dropna=True):
            col_issues.append("Case inconsistency")
        if df[col].str.strip().nunique(dropna=True) != df[col].nunique(dropna=True):
            col_issues.append("Whitespace issue")
        orig_unique = df[col].nunique(dropna=True)
        norm_unique = df[col].str.lower().str.strip().nunique(dropna=True)
        if norm_unique < orig_unique:
            col_issues.append(f"Potential duplicates (unique values drop: {orig_unique} → {norm_unique})")
        if col_issues:
            issues[col] = col_issues
    return issues

issues_found = check_text_standardization(df)
print("Columns needing standardization:\n", issues_found)


Columns needing standardization:
 {}


#### checking if any data cleaning is required in numeric columns

In [20]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# 1. Summary statistics (to check ranges, min/max, etc.)
summary_stats = df[numeric_cols].describe().T
print("Summary Statistics:\n", summary_stats, "\n")

# 2. Check for negative values
negative_values = (df[numeric_cols] < 0).sum()
print("Negative Values Count:\n", negative_values, "\n")

# 3. Check if Discount is within valid range [0, 1]
discount_issues = df[(df['Discount'] < 0) | (df['Discount'] > 1)].shape[0]
print("Invalid Discount Values (not between 0 and 1):", discount_issues, "\n")

# 4. Check if Unit_Price is less than Unit_Cost
unit_price_issues = df[df['Unit_Price'] < df['Unit_Cost']].shape[0]
print("Cases where Unit Price < Unit Cost:", unit_price_issues, "\n")


Summary Statistics:
                 count        mean          std      min        25%       50%  \
Product_ID     1000.0  1050.12800    29.573505  1001.00  1024.0000  1051.000   
Sales_Amount   1000.0  5019.26523  2846.790126   100.12  2550.2975  5019.300   
Quantity_Sold  1000.0    25.35500    14.159006     1.00    13.0000    25.000   
Unit_Cost      1000.0  2475.30455  1417.872546    60.28  1238.3800  2467.235   
Unit_Price     1000.0  2728.44012  1419.399839   167.12  1509.0850  2696.400   
Discount       1000.0     0.15239     0.087200     0.00     0.0800     0.150   

                    75%      max  
Product_ID     1075.000  1100.00  
Sales_Amount   7507.445  9989.04  
Quantity_Sold    38.000    49.00  
Unit_Cost      3702.865  4995.30  
Unit_Price     3957.970  5442.15  
Discount          0.230     0.30   

Negative Values Count:
 Product_ID       0
Sales_Amount     0
Quantity_Sold    0
Unit_Cost        0
Unit_Price       0
Discount         0
dtype: int64 

Invalid Discount V

#### Save cleaned dataset

In [21]:

cleaned_file_path = 'sales_data_cleaned.csv'
df.to_csv(cleaned_file_path, index=False)

print("Data cleaning and preprocessing completed . Cleaned file saved as:", cleaned_file_path)

Data cleaning and preprocessing completed . Cleaned file saved as: sales_data_cleaned.csv
